In [1]:
%cd src/
from pathlib import Path
import numpy as np
import math
from itertools import groupby
import h5py
import numpy as np
import unicodedata
import cv2
import torch
from torch import nn
from torchvision.models import resnet50, resnet101
from torch.autograd import Variable
import torchvision
from data import preproc as pp
from data import evaluation
from torch.utils.data import Dataset
import time
import timm



class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=128):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class OCR(nn.Module):

    def __init__(self, vocab_len, hidden_dim, nheads,
                 num_encoder_layers, num_decoder_layers):
        super().__init__()
    
#         self.backbone = timm.create_model('ecaresnet101d', pretrained=True,)
#         del self.backbone.fc
        self.backbone = resnet101(pretrained=True)
        del self.backbone.fc
        
        
#         del self.backbone.classifier, self.backbone.conv_head, self.backbone.bn2,self.backbone.act2,self.backbone.global_pool
        _ = self.backbone.to("cpu")
#         for name,p in self.backbone.named_parameters():
#             if "bn" not in name or "attnpool" in name:
#                 p.requires_grad =  False

        # create a default PyTorch transformer
        # create conversion layer
        self.conv = nn.Conv2d(2048, hidden_dim, 1)

        # create a default PyTorch transformer
        self.transformer = nn.Transformer(
            hidden_dim, nheads, num_encoder_layers, num_decoder_layers)

        # prediction heads with length of vocab
        # DETR used basic 3 layer MLP for output
        self.vocab = nn.Linear(hidden_dim,vocab_len)

        # output positional encodings (object queries)
        self.decoder = nn.Embedding(vocab_len, hidden_dim)
        self.query_pos = PositionalEncoding(hidden_dim, .2)

        # spatial positional encodings, sine positional encoding can be used.
        # Detr baseline uses sine positional encoding.
        self.row_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
        self.col_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
        self.trg_mask = None
  
    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz), 1)
        mask = mask.masked_fill(mask==1, float('-inf'))
        return mask
    
#     def get_feature(self,x):
#             x = self.backbone.conv_stem(x)
#             x = self.backbone.bn1(x)   
#             x = self.backbone.act1(x)
#             x = self.backbone.blocks(x)
#             return x

    def get_feature(self,x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)   
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        return x

#     def get_feature(self, x):
#         x = self.backbone.forward_features(x)
#         return x


    def make_len_mask(self, inp):
        return (inp == 0).transpose(0, 1)


    def forward(self, inputs, trg):
        # propagate inputs through ResNet-101 up to avg-pool layer
        x = self.get_feature(inputs)

        # convert from 2048 to 256 feature planes for the transformer
        h = self.conv(x)

        # construct positional encodings
        bs,_,H, W = h.shape
        pos = torch.cat([
            self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
            self.row_embed[:H].unsqueeze(1).repeat(1, W, 1),
        ], dim=-1).flatten(0, 1).unsqueeze(1)

        # generating subsequent mask for target
        if self.trg_mask is None or self.trg_mask.size(0) != len(trg):
            self.trg_mask = self.generate_square_subsequent_mask(trg.shape[1]).to(trg.device)

        # Padding mask
        trg_pad_mask = self.make_len_mask(trg)

        # Getting postional encoding for target
        trg = self.decoder(trg)
        trg = self.query_pos(trg)
        
        output = self.transformer(pos + 0.1 * h.flatten(2).permute(2, 0, 1), trg.permute(1,0,2), tgt_mask=self.trg_mask, 
                                  tgt_key_padding_mask=trg_pad_mask.permute(1,0))

        return self.vocab(output.transpose(0,1))


def make_model(vocab_len, hidden_dim=256, nheads=4,
                 num_encoder_layers=4, num_decoder_layers=4):
    
    return OCR(vocab_len, hidden_dim, nheads,
                 num_encoder_layers, num_decoder_layers)

"""
Uses generator functions to supply train/test with data.
Image renderings and text are created on the fly each time.
"""

class DataGenerator(Dataset):
    """Generator class with data streaming"""

    def __init__(self, source, split, transform, tokenizer):
        self.tokenizer = tokenizer
        self.transform = transform
        
        self.split = split
        self.dataset = dict()

        with h5py.File(source, "r") as f:
            self.dataset[self.split] = dict()

            self.dataset[self.split]['dt'] = np.array(f[self.split]['dt'])
            self.dataset[self.split]['gt'] = np.array(f[self.split]['gt'])
          
            randomize = np.arange(len(self.dataset[self.split]['gt']))
            np.random.seed(42)
            np.random.shuffle(randomize)

            self.dataset[self.split]['dt'] = self.dataset[self.split]['dt'][randomize]
            self.dataset[self.split]['gt'] = self.dataset[self.split]['gt'][randomize]

            # decode sentences from byte
            self.dataset[self.split]['gt'] = [x.decode() for x in self.dataset[self.split]['gt']]
            
        self.size = len(self.dataset[self.split]['gt'])


    def __getitem__(self, i):
        img = self.dataset[self.split]['dt'][i]
        
        #making image compatible with resnet
        img = np.repeat(img[..., np.newaxis],3, -1).astype("float32")   
#         img = pp.normalization(img).astype("float32")

        if self.transform is not None:
            aug = self.transform(image=img)
            img = aug['image']
            
#             img = self.transform(img)
            
        y_train = self.tokenizer.encode(self.dataset[self.split]['gt'][i]) 
        
        #padding till max length
        y_train = np.pad(y_train, (0, self.tokenizer.maxlen - len(y_train)))

        gt = torch.Tensor(y_train)

        return img, gt          

    def __len__(self):
      return self.size

class Tokenizer():
    """Manager tokens functions and charset/dictionary properties"""

    def __init__(self, chars, max_text_length=128):
        self.PAD_TK, self.UNK_TK,self.SOS,self.EOS = "¶", "¤", "SOS", "EOS"
        self.chars = [self.PAD_TK] + [self.UNK_TK ]+ [self.SOS] + [self.EOS] +list(chars)
        self.PAD = self.chars.index(self.PAD_TK)
        self.UNK = self.chars.index(self.UNK_TK)

        self.vocab_size = len(self.chars)
        self.maxlen = max_text_length

    def encode(self, text):
        """Encode text to vector"""
        text = unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("ASCII")
        text = " ".join(text.split())

        groups = ["".join(group) for _, group in groupby(text)]
        text = "".join([self.UNK_TK.join(list(x)) if len(x) > 1 else x for x in groups])
        encoded = []

        text = ['SOS'] + list(text) + ['EOS']
        for item in text:
            index = self.chars.index(item)
            index = self.UNK if index == -1 else index
            encoded.append(index)

        return np.asarray(encoded)

    def decode(self, text):
        """Decode vector to text"""
        
        decoded = "".join([self.chars[int(x)] for x in text if x > -1])
        decoded = self.remove_tokens(decoded)
        decoded = pp.text_standardize(decoded)

        return decoded

    def remove_tokens(self, text):
        """Remove tokens (PAD) from text"""

        return text.replace(self.PAD_TK, "").replace(self.UNK_TK, "")

import os
import datetime
import string

batch_size = 16
epochs = 200

# define paths
#change paths accordingly
source = 'washington'
source_path = '../data/{}.hdf5'.format(source)
output_path = os.path.join("..", "output", source)
target_path = os.path.join(output_path, "checkpoint_weights_iam_{}.hdf5".format("dsa"))
os.makedirs(output_path, exist_ok=True)

# define input size, number max of chars per line and list of valid chars
input_size = (1024, 128, 1)
max_text_length = 128
charset_base = string.printable[:95]
# charset_base = string.printable[:36].lower() + string.printable[36+26:95].lower() 

print("source:", source_path)
print("output", output_path)
print("target", target_path)
print("charset:", charset_base)

import torchvision.transforms as T
local_rank = 1
device = torch.device("cuda:{}".format(local_rank))

# transform = T.Compose([
#     T.ToTensor()])
tokenizer = Tokenizer(charset_base)
import albumentations
import albumentations.pytorch



transform_valid = albumentations.Compose(
    [
        albumentations.Normalize(),
        albumentations.pytorch.ToTensorV2()
    ]
)



/home/mhamdan/seq2seqAttenHTR/Transformer_ocr/src
source: ../data/washington.hdf5
output ../output/washington
target ../output/washington/checkpoint_weights_iam_dsa.hdf5
charset: 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 


In [2]:
tokenizer = Tokenizer(charset_base)


In [3]:
model = make_model( vocab_len=tokenizer.vocab_size,hidden_dim=256, nheads=4,
                 num_encoder_layers=4, num_decoder_layers=4)

_ = model.to(device)

In [4]:
import torch

In [6]:
d = torch.load("/home/mhamdan/seq2seqAttenHTR/Transformer_ocr/output/honest-yogurt-19/washington_firstbest.pt")

In [7]:
f = {}
for i in d:
    f[i.replace("module.","")] = d[i]

model.load_state_dict(f)

<All keys matched successfully>

In [99]:
def get_memory(model,imgs):
    x = model.conv(model.get_feature(imgs))
    bs,_,H, W = x.shape
    pos = torch.cat([
            model.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
            model.row_embed[:H].unsqueeze(1).repeat(1, W, 1),
        ], dim=-1).flatten(0, 1).unsqueeze(1)

    return model.transformer.encoder(pos +  0.1 * x.flatten(2).permute(2, 0, 1))
    

def test(model, test_loader, max_text_length,beam_size=3):
    model.eval()
    predicts = []
    gt = []
    imgs = []
    c=0
    with torch.no_grad():
        for batch in test_loader:
            break_counter=0            
            src, trg = batch
            imgs.append(src.flatten(0,1))
            src, trg = src.to(device), trg.to(device)            
            memory = get_memory(model,src.float())
            out_indexes = [[[tokenizer.chars.index('SOS')],0] ]
            final = []
            for i in range(max_text_length):            
                all_paths_local = []
                for path,proba in out_indexes[::-1]:
                    if path[-1]==tokenizer.chars.index('EOS'):
                        final.append([path, proba])
                        continue
                    mask = model.generate_square_subsequent_mask(i+1).to(device)
                    trg_tensor = torch.LongTensor(path).unsqueeze(1).to(device)
                    output = model.vocab(model.transformer.decoder(model.query_pos(model.decoder(trg_tensor)), memory,tgt_mask=mask))
                    probab, indexes = torch.topk(output.log_softmax(-1),beam_size)
                    probab, indexes = probab.flatten(), indexes.flatten()
                    
                    for k in range(beam_size):
                        words,prob = path.copy(), proba
                        words.append(int(indexes[k].cpu().numpy()))
                        prob += probab[k].cpu().numpy()
                        all_paths_local.append([words,prob])

                out_indexes = sorted(all_paths_local, reverse=False, key=lambda l: l[1])[-beam_size:]
    #             print("out",out_indexes)
#                 if break_counter:
#                     break
                if len(final)==beam_size:
                    break
            path = sorted(final, reverse=False, key=lambda l: l[1])[-1][0]
            predicts.append(tokenizer.decode(path))
            gt.append(tokenizer.decode(trg.flatten(0,1)))
    
#             if c==2:
#                 break
            c+=1
    return predicts, gt, imgs

In [10]:
test_loader = torch.utils.data.DataLoader(DataGenerator(source_path,'test',transform_valid, tokenizer), batch_size=1, shuffle=False, num_workers=6)

In [105]:
predicts, gt, imgs = test(model, test_loader, max_text_length, beam_size=5)

In [96]:
final = test(model, test_loader, max_text_length, beam_size=5)

In [101]:
# for i in range(3):
#     print(tokenizer.decode(final[i][0]), final[i][1])
#     print("-"*50)

In [106]:
predicts[:3],gt[:3]

(['SOSLmetters Orders and Instructions Becember 1755 , EOS',
  'SOSarrive at Frederic5burgh , between this andEOS',
  'SOShereof , to repaur to Winchester , where you will metEOS'],
 ['SOSletters orders and instructions december 1755 . EOS',
  'SOSarrive at fredericksburgh , between this andEOS',
  'SOShereof , to repair to winchester , where you will meetEOS'])

In [32]:
predicts[:3],gt[:3]

(['SOSLetters Orders and Instructions Becember 1755 , EOS',
  'SOSarrive at Frederichsburgh , between this andEOS',
  'SOShereof , to repar to Winchester , where you will metEOS'],
 ['SOSletters orders and instructions december 1755 . EOS',
  'SOSarrive at fredericksburgh , between this andEOS',
  'SOShereof , to repair to winchester , where you will meetEOS'])

In [107]:
predicts = list(map(lambda x : x.replace('SOS','').replace('EOS',''),predicts))
gt = list(map(lambda x : x.replace('SOS','').replace('EOS',''),gt))

evaluate = evaluation.ocr_metrics(predicts=predicts,
                                  ground_truth=gt,)

In [108]:
evaluate

array([0.26811312, 0.38611692, 0.91772152])

In [12]:
evaluate

array([0.0789446 , 0.22282633, 0.81012658])

In [11]:
evaluate

array([0.08521912, 0.24903093, 0.86245614])

In [13]:
predicts[:5]

['Letters Orders and Instructions Becember 1755 , ',
 'arrive at Frederichsburgh , between this and',
 'hereof , to repar to Winchester , where you will met',
 'at unfit for Duty , on Review ; are ordered to be',
 'mediately advertising , and sending a party or']

In [17]:
import wandb

In [18]:
    run = wandb.init(
        config={"g":1},
        project="pretrained",
    )


wandb: Currently logged in as: himanshu13 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [35]:
run.name


'curious-water-1'

In [21]:
source = "../data/rimes.hdf5"
split = "test"

In [22]:
dataset = dict()

with h5py.File(source, "r") as f:
    dataset[split] = dict()

    dataset[split]['dt'] = np.array(f[split]['dt'])
    dataset[split]['gt'] = np.array(f[split]['gt'])

    randomize = np.arange(len(dataset[split]['gt']))
    np.random.seed(42)
    np.random.shuffle(randomize)

    dataset[split]['dt'] = dataset[split]['dt'][randomize]
    dataset[split]['gt'] = dataset[split]['gt'][randomize]

    # decode sentences from byte
    dataset[split]['gt'] = [x.decode() for x in dataset[split]['gt']]


In [25]:
dataset["test"]["gt"][2]

'Merci de faire le necessaire'

In [69]:
img = cv2.imread("../raw/iam/lines/b01/b01-000/b01-000-00.png")

In [70]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [65]:
img.shape, gray.shape

((140, 1856, 3), (140, 1856))

In [58]:
im = pp.preprocess("../raw/iam/lines/a04/a04-000/a04-000-00.png", (1024, 128, 1))

In [71]:
imm = pp.remove_cursive_style(gray)

In [72]:
cv2.imwrite("temp.png",imm)

True

In [54]:
im.shape

(1024, 128)